In [1]:
from bert_score import BERTScorer

model_type = "facebook/bart-large"
model_lang = "en"
# load scorer
bert_scorer = BERTScorer(model_type=model_type, lang=model_lang)

In [2]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [8]:
import random
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/hdu5/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
from datasets import load_dataset, load_metric
raw_datasets = load_dataset('csv', data_files={'train': 'AutoReview_summarized.csv'
                                            #    'validation': 'dataset/all_data_val/all_data_val.csv',
                                            })
print(raw_datasets)

Using custom data configuration default
Reusing dataset csv (/home/hdu5/.cache/huggingface/datasets/csv/default-4327c2f5a178f0c7/0.0.0/2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2)


DatasetDict({
    train: Dataset({
        features: ['pid', 'document', 'summary'],
        num_rows: 484
    })
})


In [5]:
import pandas as pd
import numpy as np
DF = pd.DataFrame(columns=['pid', 'document', 'summary', 'classification'])
cnt_dict = {'pid':[], 'cnt':[]}

In [6]:
# get each sentence from the original summary
for item in raw_datasets['train'] : 
    # print(item['summary'])
    # res = item['summary'].split('.')
    res = nlp(item['summary'])
    cnt = 0 # number of positive examples
    for sent in res.sents :
        # print(sent)
        str_summary = str(sent)
        str_pid = item['pid']
        str_doc = item['document']
        str_classification = '1' # those are positive test cases
        # print(pd.Series([str_pid, str_doc, str_summary], index=DF.columns))
        cnt = cnt + 1
        DF = DF.append(pd.Series([str_pid, str_doc, str_summary, str_classification], index=DF.columns), ignore_index=True)
        # break
    cnt_dict['pid'].append(str_pid)
    cnt_dict['cnt'].append(cnt)
    

In [15]:
def generate_random_negative(index, positive_examples_lst) :
    while True :
        random_index = random.randint(0, len(cnt_dict['pid'])-1)
        if random_index != index :
            break
    
    random_pid = cnt_dict['pid'][random_index]
    for i in raw_datasets['train'] :
        if i['pid'] == random_pid :
            # print('llll')
            random_from_raw = i
            break
    # print(random_pid)
    # raw_datasets['train'][]
    # print(random_from_raw['summary'])
    res = nlp(random_from_raw['summary'])
    # print(res)
    str_summary_lst = []
    for sent in res.sents :
        str_summary_lst.append(str(sent))
    random_sent = str_summary_lst[random.randint(0, len(str_summary_lst) - 1)]
    cmp_lst = []
    
    # positive_examples_lst = positive_examples['summary'].values.tolist()
    for i in range(len(positive_examples_lst)) :
        cmp_lst.append(random_sent)
    # print("asdf: "+cmp_lst[0])
    return cmp_lst

In [16]:
cur_LOC = 0
F1_lst = []
threshold = 0.45
# display(f) # display the bar


neg_DF = pd.DataFrame(columns=['pid', 'document', 'summary', 'classification'])
# generating negative examples from positive examples
for i in range(len(cnt_dict['pid'])) :
    # print (i)
    num_of_positive = cnt_dict['cnt'][i]
    # print(num_of_positive)
    positive_examples = DF.loc[cur_LOC:num_of_positive]
    # print(positive_examples)
    positive_examples_lst = positive_examples['summary'].values.tolist()
    # generate a random index
    cmp_lst = generate_random_negative(i, positive_examples_lst)
    while (not cmp_lst) or len(cmp_lst[0]) < 30 :
        cmp_lst = generate_random_negative(i, positive_examples_lst)
    # if len(str_summary_lst) > len(positive_examples_lst) :
    #     str_summary_lst_copy = str_summary_lst.copy()
    #     str_summary_lst = str_summary_lst[:len(positive_examples_lst)]

    # while len(str_summary_lst) < len(positive_examples_lst) :
    #     str_summary_lst.append(str_summary_lst[-1])
        
    P, R, F1 = bert_scorer.score(positive_examples_lst, cmp_lst)
    attempts = 0
    print("case"+str(i)+": attempt"+str(attempts))
    while np.average(F1.numpy()) > threshold :
        print("└------case"+str(i)+" attempts: "+str(attempts))
        cmp_lst = generate_random_negative(i, positive_examples_lst)
        while (not cmp_lst) and len(cmp_lst[0]) < 30:
            cmp_lst = generate_random_negative(i, positive_examples_lst)
        P, R, F1 = bert_scorer.score(positive_examples_lst, cmp_lst)
        attempts = attempts + 1

    for examples in cmp_lst :
        str_pid = DF.loc[cur_LOC]['pid']
        str_doc = DF.loc[cur_LOC]['document']
        str_summary = cmp_lst[0]
        str_classification = '0'
        neg_DF = neg_DF.append(pd.Series([str_pid, str_doc, str_summary, str_classification], index=DF.columns), ignore_index=True)
    F1_lst.append(np.average(F1.numpy()))
    cur_LOC = cur_LOC + num_of_positive
    # f.value += 1

case0: attempt0
└------case0 attempts: 0
└------case0 attempts: 1
└------case0 attempts: 2
└------case0 attempts: 3
└------case0 attempts: 4
└------case0 attempts: 5
└------case0 attempts: 6
└------case0 attempts: 7
└------case0 attempts: 8
└------case0 attempts: 9
└------case0 attempts: 10
└------case0 attempts: 11
└------case0 attempts: 12


KeyboardInterrupt: 

In [ ]:
random_from_raw['summary']

"The class hierarchy didn't need to be included in this document.  Not sure why it was.\nThe wiki is supposed to explain why you've written what you wrote.  But you seem to have copied the code and inserted only a line or two, not much more than in the comments in the code file."

In [7]:
DF.to_csv('positive.csv', index=False)

In [ ]:
F1_lst[1]